In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/top-rated-movies-dataset/Movies_dataset.csv


In [2]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings("ignore")


2026-01-21 04:34:28.627130: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768970068.830051      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768970068.889112      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768970069.388690      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768970069.388737      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768970069.388740      24 computation_placer.cc:177] computation placer alr

In [3]:
df = pd.read_csv("/kaggle/input/top-rated-movies-dataset/Movies_dataset.csv")

df = df[['original_title', 'genre', 'overview']]
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df.shape)
df.head()


(9996, 3)


,original_title,genre,overview
0,The Shawshank Redemption,"Drama, Crime",Imprisoned in the 1940s for the double murder ...
1,The Godfather,"Drama, Crime","Spanning the years 1945 to 1955, a chronicle o..."
2,The Godfather Part II,"Drama, Crime",In the continuing saga of the Corleone crime f...
3,Schindler's List,"Drama, History, War",The true story of how businessman Oskar Schind...
4,12 Angry Men,Drama,The defense and the prosecution have rested an...


In [4]:
df['genre'] = df['genre'].str.replace('|', ' ', regex=False)
df['content'] = df['genre'] + " " + df['overview']

df['content'] = df['content'].str.lower()


In [5]:
tfidf = TfidfVectorizer(
    stop_words='english',
    max_features=5000
)

X = tfidf.fit_transform(df['content']).toarray()

print("TF-IDF Shape:", X.shape)


TF-IDF Shape: (9996, 5000)


In [6]:
input_dim = X.shape[1]

input_layer = Input(shape=(input_dim,))
encoded = Dense(1024, activation='relu')(input_layer)
encoded = Dense(256, activation='relu')(encoded)

decoded = Dense(1024, activation='relu')(encoded)
output_layer = Dense(input_dim, activation='sigmoid')(decoded)

autoencoder = Model(input_layer, output_layer)
encoder = Model(input_layer, encoded)

autoencoder.summary()


I0000 00:00:1768970082.498809      24 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1768970082.502678      24 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 5000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     5,121,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       262,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1024)           │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5000)           │     5,125,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,771,592 (41.09 MB)

 Trainable params: 10,771,592 (41.09 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
autoencoder.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse'
)

autoencoder.fit(
    X, X,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)


Epoch 1/20


I0000 00:00:1768970086.059353      69 service.cc:152] XLA service 0x7e9e64009a30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1768970086.059391      69 service.cc:160]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1768970086.059396      69 service.cc:160]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1768970086.354271      69 cuda_dnn.cc:529] Loaded cuDNN version 91002


 42/282 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1702

I0000 00:00:1768970087.479232      69 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0487 - val_loss: 2.0000e-04
Epoch 2/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2.0000e-04 - val_loss: 2.0000e-04
Epoch 3/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2.0000e-04 - val_loss: 2.0000e-04
Epoch 4/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2.0000e-04 - val_loss: 2.0000e-04
Epoch 5/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2.0000e-04 - val_loss: 2.0000e-04
Epoch 6/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2.0000e-04 - val_loss: 2.0000e-04
Epoch 7/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2.0000e-04 - val_loss: 2.0000e-04
Epoch 8/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2.0000e-04 - val_loss: 2.0000e-04
Epoch 9/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2.0000e-04 - val_loss: 2.0000e-04
Epoch 10/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2.0000e-04 - val_loss: 2.0000e-04
Epoch 11/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2.0000e-04 - 

In [8]:
movie_embeddings = encoder.predict(X)
print(movie_embeddings.shape)


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
(9996, 256)


In [9]:
cosine_sim_nn = cosine_similarity(movie_embeddings)


In [10]:
def recommend_movie(movie_name, top_n=5):
    movie_name = movie_name.lower()
    titles = df['original_title'].str.lower()

    if movie_name not in titles.values:
        return "Movie not found"

    idx = titles[titles == movie_name].index[0]

    sim_scores = list(enumerate(cosine_sim_nn[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    top_movies = sim_scores[1:top_n+1]

    return [df.iloc[i[0]]['original_title'] for i in top_movies]


In [11]:
recommend_movie("Inception")


['Harold and Maude', 'Cypher', 'Jack the Giant Slayer', 'RED', 'Hellboy']

In [12]:
recommend_movie("The Dark Knight")


['Batman: The Long Halloween, Part One',
 'Batman: The Long Halloween, Part Two',
 'Star Trek Into Darkness',
 'Batman: The Dark Knight Returns, Part 1',
 'ドラゴンボール 神龍の伝説']

In [13]:
recommend_movie("transformer")

'Movie not found'